In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import xarray as xr

In [2]:
metrics = xr.open_dataset("annual_mean_metrics.nc")

In [3]:
def _ttest_ind_wrapper(a, b, axis=0, equal_var=True, nan_policy="propagate"):
    result = scipy.stats.ttest_ind(
        a,
        b,
        axis=axis,
        equal_var=equal_var,
        nan_policy=nan_policy
    )
    return result.statistic, result.pvalue


def xarray_t_test(
    a,
    b,
    dim,
    equal_var=True,
    nan_policy="propagate",
):
    kwargs = {
        "axis": -1,
        "equal_var": equal_var,
        "nan_policy": nan_policy,
    }
    return xr.apply_ufunc(
        _ttest_ind_wrapper,
        a,
        b,
        input_core_dims=[[dim], [dim]],
        output_core_dims=[[], []],
        kwargs=kwargs,
        exclude_dims={dim}
    )

In [4]:
VARIABLES = ["total_precipitation_rate",  "surface_temperature", "net_surface_radiative_flux"]
REGIONS = ["land", "ocean/sea-ice"]


def metrics_table(metrics, configuration, validation_year):
    metrics = metrics.sel(validation_year=validation_year).drop("validation_year")
    stacked_metrics = metrics.stack(sample=("configuration", "time")).drop("samples")
    stacked_metrics = stacked_metrics.dropna("sample")
    
    in_five_year_window = (stacked_metrics.time.dt.year.isin(range(2018, 2023)))
    baseline_samples = (stacked_metrics.configuration == "Baseline") & in_five_year_window
    configuration_samples = (stacked_metrics.configuration == configuration) & in_five_year_window
    
    t_statistic, p_values = xarray_t_test(
        stacked_metrics.isel(sample=configuration_samples),
        stacked_metrics.isel(sample=baseline_samples),
        "sample",
        equal_var=False,
        nan_policy="omit"
    )
    mean_metrics = xr.concat(
        [
            stacked_metrics.sel(sample=baseline_samples).mean("sample"),
            stacked_metrics.sel(sample=configuration_samples).mean("sample")
        ],
        dim=pd.Index(["Baseline", configuration], name="configuration")
    )
    significant = p_values < 0.05
    insignificant = p_values >= 0.05
    
    unmasked_metrics = (
        mean_metrics[VARIABLES]
        .sel(region=REGIONS)
        .to_array()
        .rename("metric")
        .to_dataframe(dim_order=["variable", "metric", "region", "configuration", "climate"])
        .unstack(level=-1)
        .unstack(level=-1)
    )
    masked_metrics = (
        mean_metrics[VARIABLES]
        .where(significant[VARIABLES])
        .sel(region=REGIONS)
        .to_array()
        .rename("metric")
        .to_dataframe(dim_order=["variable", "metric", "region", "configuration", "climate"])
        .unstack(level=-1)
        .unstack(level=-1)
    )
    return mean_metrics, significant[VARIABLES], unmasked_metrics, masked_metrics

In [5]:
mean_metrics_year_one, significant_year_one, unmasked_year_one, masked_year_one = metrics_table(metrics, "ML-corrected seed 2", "Year one")
mean_metrics_year_two, significant_year_two, unmasked_year_two, masked_year_two = metrics_table(metrics, "ML-corrected seed 2", "Year two")

### Source of data in Table 2 of the manuscript

In [6]:
unmasked_year_two.round(decimals=1)

metric                      \
climate                                         Minus 4 K                       
configuration                                    Baseline ML-corrected seed 2   
variable                   metric region                                        
total_precipitation_rate   mean   land               -0.8                -0.2   
                                  ocean/sea-ice       0.1                -0.1   
                           rmse   land                1.7                 1.2   
                                  ocean/sea-ice       1.2                 1.2   
                           mae    land                1.1                 0.7   
                                  ocean/sea-ice       0.7                 0.7   
surface_temperature        mean   land                0.3                 0.2   
                                  ocean/sea-ice       0.0                 0.1   
                           rmse   land                3.6                 2.8   
                                  ocean/sea-ice       0.7                 0.6   
                           mae    land                2.8                 2.0   
                                  ocean/sea-ice       0.2                 0.2   
net_surface_radiative_flux mean   land              -10.2                -6.0   
                                  ocean/sea-ice      -1.3                -3.2   
                           rmse   land               21.7                15.3   
                                  ocean/sea-ice       8.9                 9.6   
                           mae    land               14.7                11.2   
                                  ocean/sea-ice       6.1                 7.0   

                                                             \
climate                                         Unperturbed   
configuration                                      Baseline   
variable                   metric region                      
total_precipitation_rate   mean   land                 -0.8   
                                  ocean/sea-ice         0.1   
                           rmse   land                  1.9   
                                  ocean/sea-ice         1.6   
                           mae    land                  1.1   
                                  ocean/sea-ice         0.9   
surface_temperature        mean   land                  0.4   
                                  ocean/sea-ice        -0.0   
                           rmse   land                  3.5   
                                  ocean/sea-ice         0.7   
                           mae    land                  2.7   
                                  ocean/sea-ice         0.2   
net_surface_radiative_flux mean   land                -12.9   
                                  ocean/sea-ice        -1.7   
                           rmse   land                 24.8   
                                  ocean/sea-ice         9.6   
                           mae    land                 16.4   
                                  ocean/sea-ice         6.5   

                                                                              \
climate                                                             Plus 4 K   
configuration                                   ML-corrected seed 2 Baseline   
variable                   metric region                                       
total_precipitation_rate   mean   land                         -0.2     -0.7   
                                  ocean/sea-ice                -0.1      0.1   
                           rmse   land                          1.4      1.8   
                                  ocean/sea-ice                 1.6      1.8   
                           mae    land                          0.8      1.1   
                                  ocean/sea-ice                 1.0      1.1   
surface_temperature        mean   land                         -0.1      0.1   
 

In [7]:
masked_year_two.round(decimals=1)

metric                      \
climate                                         Minus 4 K                       
configuration                                    Baseline ML-corrected seed 2   
variable                   metric region                                        
total_precipitation_rate   mean   land               -0.8                -0.2   
                                  ocean/sea-ice       0.1                -0.1   
                           rmse   land                1.7                 1.2   
                                  ocean/sea-ice       NaN                 NaN   
                           mae    land                1.1                 0.7   
                                  ocean/sea-ice       NaN                 NaN   
surface_temperature        mean   land                NaN                 NaN   
                                  ocean/sea-ice       0.0                 0.1   
                           rmse   land                3.6                 2.8   
                                  ocean/sea-ice       0.7                 0.6   
                           mae    land                2.8                 2.0   
                                  ocean/sea-ice       0.2                 0.2   
net_surface_radiative_flux mean   land              -10.2                -6.0   
                                  ocean/sea-ice      -1.3                -3.2   
                           rmse   land               21.7                15.3   
                                  ocean/sea-ice       8.9                 9.6   
                           mae    land               14.7                11.2   
                                  ocean/sea-ice       6.1                 7.0   

                                                             \
climate                                         Unperturbed   
configuration                                      Baseline   
variable                   metric region                      
total_precipitation_rate   mean   land                 -0.8   
                                  ocean/sea-ice         0.1   
                           rmse   land                  1.9   
                                  ocean/sea-ice         NaN   
                           mae    land                  1.1   
                                  ocean/sea-ice         NaN   
surface_temperature        mean   land                  0.4   
                                  ocean/sea-ice        -0.0   
                           rmse   land                  3.5   
                                  ocean/sea-ice         0.7   
                           mae    land                  2.7   
                                  ocean/sea-ice         0.2   
net_surface_radiative_flux mean   land                -12.9   
                                  ocean/sea-ice        -1.7   
                           rmse   land                 24.8   
                                  ocean/sea-ice         NaN   
                           mae    land                 16.4   
                                  ocean/sea-ice         6.5   

                                                                              \
climate                                                             Plus 4 K   
configuration                                   ML-corrected seed 2 Baseline   
variable                   metric region                                       
total_precipitation_rate   mean   land                         -0.2     -0.7   
                                  ocean/sea-ice                -0.1      0.1   
                           rmse   land                          1.4      1.8   
                                  ocean/sea-ice                 NaN      1.8   
                           mae    land                          0.8      1.1   
                                  ocean/sea-ice                 NaN      NaN   
surface_temperature        mean   land                         -0.1      0.1   
 

In [8]:
def identical_sign(ds, dim):
    """Check if variables have the same sign for all values along a dimension"""
    return (ds > 0).all(dim) | (ds < 0).all(dim)

### To measure improvement of the RMSE we compute minus the percent deviation from the baseline

$$-100 \cdot \frac{M_{ML} - M_{baseline}}{M_{baseline}}$$

In the manuscript we only report cases where the percent improvement (or degredation) was statistically significant and of a consistent sign in all climates.

In [9]:
def compute_rmse_percent_improvement(metrics, significant, configuration):
    improvement = -(100 * (metrics.sel(configuration=configuration) - metrics.sel(configuration="Baseline")) / metrics.sel(configuration="Baseline")).drop("configuration")
    improvement_min = improvement.min("climate")
    improvement_max = improvement.max("climate")
    ds = xr.concat([improvement_min, improvement_max], dim=pd.Index(["min", "max"], name="bound")).sel(metric="rmse").drop("metric")
    ds = ds.where(significant.sel(metric="rmse").drop("metric").all("climate") & identical_sign(ds, "bound"))
    return ds[VARIABLES].sel(region=REGIONS).to_array().rename("metric").to_dataframe(dim_order=["variable", "region", "bound"]).unstack(level=-1)

In [10]:
compute_rmse_percent_improvement(mean_metrics_year_two, significant_year_two, "ML-corrected seed 2").round(decimals=1)

metric      
bound                                       min   max
variable                   region                    
total_precipitation_rate   land            16.6  29.6
                           ocean/sea-ice    NaN   NaN
surface_temperature        land            20.4  22.6
                           ocean/sea-ice   16.3  20.7
net_surface_radiative_flux land            29.8  51.1
                           ocean/sea-ice    NaN   NaN

### To measure improvement of the bias we compute minus the percent deviation from the baseline for the absolute values

$$-100 \cdot \frac{|M_{ML}| - |M_{baseline}|}{|M_{baseline}|}$$

In the manuscript we only report cases where the percent improvement (or degredation) was statistically significant and of a consistent sign in all climates.

In [11]:
def compute_bias_percent_improvement(metrics, significant, configuration):
    improvement = -(100 * (np.abs(metrics.sel(configuration=configuration)) - np.abs(metrics.sel(configuration="Baseline"))) / np.abs(metrics.sel(configuration="Baseline"))).drop("configuration")
    improvement_min = improvement.min("climate")
    improvement_max = improvement.max("climate")
    ds = xr.concat([improvement_min, improvement_max], dim=pd.Index(["min", "max"], name="bound")).sel(metric="mean").drop("metric")
    ds = ds.where(significant.sel(metric="mean").drop("metric").all("climate") & identical_sign(ds, "bound"))
    return ds[VARIABLES].sel(region=REGIONS).to_array().rename("metric").to_dataframe(dim_order=["variable", "region", "bound"]).unstack(level=-1)

In [12]:
compute_bias_percent_improvement(mean_metrics_year_two, significant_year_two, "ML-corrected seed 2").round(decimals=1)

metric      
bound                                       min   max
variable                   region                    
total_precipitation_rate   land            76.4  99.9
                           ocean/sea-ice    NaN   NaN
surface_temperature        land             NaN   NaN
                           ocean/sea-ice -491.0 -54.5
net_surface_radiative_flux land            41.0  87.3
                           ocean/sea-ice    NaN   NaN

### Impact of using the first year of the fine-resolution run as validation data

In [13]:
unmasked_year_one.round(decimals=1)

metric                      \
climate                                         Minus 4 K                       
configuration                                    Baseline ML-corrected seed 2   
variable                   metric region                                        
total_precipitation_rate   mean   land               -0.7                -0.1   
                                  ocean/sea-ice       0.1                -0.1   
                           rmse   land                1.7                 1.2   
                                  ocean/sea-ice       1.3                 1.1   
                           mae    land                1.0                 0.7   
                                  ocean/sea-ice       0.8                 0.7   
surface_temperature        mean   land                0.2                 0.1   
                                  ocean/sea-ice       0.0                 0.1   
                           rmse   land                3.6                 2.9   
                                  ocean/sea-ice       0.7                 0.6   
                           mae    land                2.8                 2.1   
                                  ocean/sea-ice       0.2                 0.2   
net_surface_radiative_flux mean   land               -9.7                -5.5   
                                  ocean/sea-ice      -1.6                -3.5   
                           rmse   land               20.7                15.1   
                                  ocean/sea-ice       9.0                 9.6   
                           mae    land               14.2                11.1   
                                  ocean/sea-ice       6.2                 7.1   

                                                             \
climate                                         Unperturbed   
configuration                                      Baseline   
variable                   metric region                      
total_precipitation_rate   mean   land                 -0.8   
                                  ocean/sea-ice         0.1   
                           rmse   land                  1.7   
                                  ocean/sea-ice         1.5   
                           mae    land                  1.1   
                                  ocean/sea-ice         0.9   
surface_temperature        mean   land                  0.3   
                                  ocean/sea-ice        -0.0   
                           rmse   land                  3.4   
                                  ocean/sea-ice         0.7   
                           mae    land                  2.6   
                                  ocean/sea-ice         0.2   
net_surface_radiative_flux mean   land                -12.7   
                                  ocean/sea-ice        -1.8   
                           rmse   land                 23.6   
                                  ocean/sea-ice         9.2   
                           mae    land                 15.8   
                                  ocean/sea-ice         6.3   

                                                                              \
climate                                                             Plus 4 K   
configuration                                   ML-corrected seed 2 Baseline   
variable                   metric region                                       
total_precipitation_rate   mean   land                         -0.1     -0.7   
                                  ocean/sea-ice                -0.2      0.1   
                           rmse   land                          1.3      1.7   
                                  ocean/sea-ice                 1.8      1.8   
                           mae    land                          0.8      1.1   
                                  ocean/sea-ice                 1.0      1.1   
surface_temperature        mean   land                         -0.2     -0.0   
 

In [14]:
masked_year_one.round(decimals=1)

metric                      \
climate                                         Minus 4 K                       
configuration                                    Baseline ML-corrected seed 2   
variable                   metric region                                        
total_precipitation_rate   mean   land               -0.7                -0.1   
                                  ocean/sea-ice       0.1                -0.1   
                           rmse   land                1.7                 1.2   
                                  ocean/sea-ice       1.3                 1.1   
                           mae    land                1.0                 0.7   
                                  ocean/sea-ice       0.8                 0.7   
surface_temperature        mean   land                NaN                 NaN   
                                  ocean/sea-ice       0.0                 0.1   
                           rmse   land                3.6                 2.9   
                                  ocean/sea-ice       0.7                 0.6   
                           mae    land                2.8                 2.1   
                                  ocean/sea-ice       0.2                 0.2   
net_surface_radiative_flux mean   land               -9.7                -5.5   
                                  ocean/sea-ice      -1.6                -3.5   
                           rmse   land               20.7                15.1   
                                  ocean/sea-ice       9.0                 9.6   
                           mae    land               14.2                11.1   
                                  ocean/sea-ice       6.2                 7.1   

                                                             \
climate                                         Unperturbed   
configuration                                      Baseline   
variable                   metric region                      
total_precipitation_rate   mean   land                 -0.8   
                                  ocean/sea-ice         0.1   
                           rmse   land                  1.7   
                                  ocean/sea-ice         1.5   
                           mae    land                  1.1   
                                  ocean/sea-ice         0.9   
surface_temperature        mean   land                  0.3   
                                  ocean/sea-ice        -0.0   
                           rmse   land                  3.4   
                                  ocean/sea-ice         0.7   
                           mae    land                  2.6   
                                  ocean/sea-ice         0.2   
net_surface_radiative_flux mean   land                -12.7   
                                  ocean/sea-ice        -1.8   
                           rmse   land                 23.6   
                                  ocean/sea-ice         9.2   
                           mae    land                 15.8   
                                  ocean/sea-ice         6.3   

                                                                              \
climate                                                             Plus 4 K   
configuration                                   ML-corrected seed 2 Baseline   
variable                   metric region                                       
total_precipitation_rate   mean   land                         -0.1     -0.7   
                                  ocean/sea-ice                -0.2      0.1   
                           rmse   land                          1.3      1.7   
                                  ocean/sea-ice                 1.8      1.8   
                           mae    land                          0.8      1.1   
                                  ocean/sea-ice                 1.0      1.1   
surface_temperature        mean   land                         -0.2     -0.0   
 

In [15]:
compute_rmse_percent_improvement(mean_metrics_year_one, significant_year_one, "ML-corrected seed 2").round(decimals=1)

metric      
bound                                       min   max
variable                   region                    
total_precipitation_rate   land             9.2  26.4
                           ocean/sea-ice    NaN   NaN
surface_temperature        land            19.5  28.6
                           ocean/sea-ice    NaN   NaN
net_surface_radiative_flux land            26.8  47.0
                           ocean/sea-ice    NaN   NaN

In [16]:
compute_bias_percent_improvement(mean_metrics_year_one, significant_year_one, "ML-corrected seed 2").round(decimals=1)

metric      
bound                                        min   max
variable                   region                     
total_precipitation_rate   land             82.7  99.2
                           ocean/sea-ice     NaN   NaN
surface_temperature        land              NaN   NaN
                           ocean/sea-ice -3362.8  -4.4
net_surface_radiative_flux land             43.3  95.7
                           ocean/sea-ice     NaN   NaN